#### Problem statement: 
##### To predict the quality of wine
    

#### Importing the need libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

**Loading the dataset**

In [2]:
data = pd.read_csv('dataset.csv')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


#### Checking the info of the dataset

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


From the above,it is observed that the all the features are numeric and no null values is present, and the target feature is quality feature

#### Checking the descriptive statistic of the dataset

In [5]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


Plotting the pairplot of each pair of features in the data

In [ ]:
plt.figure( figsize = (7,7) )
sns.pairplot(data)
plt.show()

<Figure size 504x504 with 0 Axes>

In [ ]:
plt.figure(figsize = (4,2.5))
cols = data.columns[:-1]
for col in cols: 
    sns.boxplot(x = col, data = data)
    plt.xlabel(f'X-axis ---> {col}')
    plt.show()

In [ ]:
cols = cols[:-1]
cols

In [ ]:
def outlier_replacer(data, col):
    first_quartile = data[col].quantile(.25)
    third_quartile = data[col].quantile(.75)
    iqr = (third_quartile - first_quartile) * 1.5
    maxim, minim = third_quartile + iqr , first_quartile - iqr
    max_index = data[col][data[col] > maxim].index
    min_index = data[col][data[col] < minim].index
    data[col].iloc[max_index] = maxim
    data[col].iloc[min_index] = minim
   

In [ ]:
for col in cols:
    outlier_replacer(data, col)

In [ ]:
for col in cols: 
    plt.figure(figsize = (4,4))
    sns.boxplot(data[col])
    plt.show()

#### Checking and removal of duplicate values

In [ ]:
data.duplicated().sum()

**removing the outliers** 

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.duplicated().sum()

#### Binning the dependent variable

In [ ]:
data.head()

In [ ]:
data.quality.unique()

In [ ]:
# 3 & 4 --> Low quality Wine.
# 5 & 6 ----- Average qualty wine. 
# 7 & 8 -----> High Quality wine.

def replace_quality(val):
    if(3 <= val <= 4 ):
        return 'Low'
    elif(5 <= val <= 6):
        return 'Average'
    elif(val >= 7):
        return 'High'

In [ ]:
data.quality =  data.quality.apply(replace_quality)

In [ ]:
data.quality.unique()

#### Splitting the dataset into dependent and independent variable

In [ ]:
X= data.iloc[:, :-1]
y = data.iloc[:, -1]

#### Scaling the independent variable

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

####  Splitting the dataset into the train  and tesing dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

**Scaling the dataset using the standard scaler**

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

#### Preparing the model for the grid search 

In [ ]:
models = {
    
    'Logistic Regression': {
        'model': LogisticRegression(),
        'hyper_parameters' : {
            'C' : list(range(1,6)),
            'fit_intercept' : [True, False],
        }
    }, # End of a model
    
    "XG Boost":{
        "model":XGBClassifier(),
        "hyper_parameters":{
            "n_estimators":[10, 20, 30, 40, 50, 100]
        }
        }, #End of a model.
    
    'Extra Tree Classifier' : {
        'model' : ExtraTreesClassifier(),
        'hyper_parameters' : {
            'n_estimators' : list(range(30,60, 100)),
             'criterion' : ['gini', 'entropy']
        }
    }, #End of a model
    
    'SVM': {
            'model':SVC(),
            'hyper_parameters': {
                'C':list(range(1,6)),
                'kernel':['rbf','poly'],
                'degree': list(range(1,5))
            }
    }, #End of a model.
    
    'RandomForest' : {
        'model' : RandomForestClassifier(),
        'hyper_parameters' : {
            'n_estimators' : [50, 70, 90, 110],
            'criterion' : ['gini', 'criterion']
        }
    }#End of a model 
           
}  # End of all the models 

In [ ]:
from sklearn.model_selection import StratifiedKFold
cv =  StratifiedKFold(n_splits = 10)

In [ ]:
predict = pd.DataFrame(score)

predict.sort_values(by='Best Score')

#### Best model is Random Forest classifier with 84.33 and the hyper parameters were  
#### criterion = gini, n_estimator = 50

In [ ]:
model = RandomForestClassifier(criterion = 'gini', n_estimators = 50)

In [ ]:
model.fit(X_train , y_train)
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score , confusion_matrix

In [ ]:
test_accuracy = accuracy_score(y_test , y_pred)
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f'Training accuracy ===> {train_accuracy}')
print(f'Testing accuracy ====> {test_accuracy}')

Checking for the best features present in the dataset

In [ ]:
best = model.feature_importances_
print(len(best))
print(best)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print(report)